In [31]:
import pandas as pd
import numpy as np
import shapely as shp
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import operator
import ast
from shapely.ops import unary_union

In [32]:
data = pd.read_excel("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/owner/base_owner/Base_owner_wise_without_geometry.xlsx")
data

,Unnamed: 0,OWNER_ID,Owner Name,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),crop_maxA
0,0,OWNER00001,001Hardy Limited,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,7.688452,"['Grass', 'Spring Oats', 'Spring Field beans']"
1,1,OWNER00002,07737281 Limited,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,10.730689,"['Winter Field beans', 'Grass', 'Lucerne']"
2,2,OWNER00003,1 Grosvenor Holdings Limited,10332472,"263 Dunstable Road, Luton LU4 8BP",BEDFORD,BEDFORD,NaN,NaN,NaN,NaN,1.636863,['Grass']
3,3,OWNER00004,106 Properties Group Limited,9789625,"1-2 The Barn, West Stoke Road, West Lavant, Ch...",CHICHESTER,WEST SUSSEX,PO20 1LJ,NaN,NaN,NaN,0.591174,['Fallow Land']
4,4,OWNER00005,112680647,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,19.016511,"['Trees and Scrubs, short Woody plants, hedger..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58000,58000,OWNER58001,Zurich Developments Limited,9133879,"246 Hawkes Mill Lane, Allesley, Coventry CV5 9FJ",COVENTRY,WEST MIDLANDS,CV5 9FJ,NaN,NaN,NaN,0.215044,['Winter Field beans']
58001,58001,OWNER58002,Zym Capital Limited,14747989,"Aquinas House, 63 Warstone Lane, Birmingham B1...",STRATFORD-ON-AVON,WARWICKSHIRE,B94 5DT,NaN,NaN,NaN,4.174537,['Spring Oats']
58002,58002,OWNER58003,Zymshire Limited,NaN,NaN,NaN,NaN,NaN,NaN,Y,ZYMSHIRE LIMITED,71.614820,"['Winter Wheat', 'Trees and Scrubs, short Wood..."
58003,58003,OWNER58004,Zyox Limited,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,Y,ZYOX LIMITED,132.221048,"['Winter Wheat', 'Spring Wheat', 'Grass', 'Spr..."


In [33]:
sic_code = pd.read_excel("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/LR/Companieshouse.xlsx",sheet_name="Companies_UK_land_ag")
sic_code

,company_name,company_number,company_status,company_type,company_subtype,dissolution_date,incorporation_date,removed_date,registered_date,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
0,@ CLEEVE HILL LIMITED,11723541,Active,Private limited company,NaN,NaN,2018-12-12 00:00:00,NaN,NaN,01500 55900 70229,Upper Lodge Cleeve Hill Kelston Road Bath BA1 9AD,1500,Mixed farming,55900.0,Other accommodation,70229.0,Management consultancy activities other than f...,NaN,NaN
1,1 MORE TREE COMMUNITY INTEREST COMPANY,11038379,Active,Private limited company,Community Interest Company (CIC),NaN,2017-10-30 00:00:00,NaN,NaN,1300,12 Ranelagh Road Wellingborough NN8 1HZ,1300,Plant propagation,NaN,NaN,NaN,NaN,NaN,NaN
2,1 ONION LIMITED,3354877,Active,Private limited company,NaN,NaN,1997-04-17 00:00:00,NaN,NaN,01250 62090,White Acre Pound Lane Meonstoke SO32 3NP,1250,Growing of other tree and bush fruits and nuts,62090.0,Other information technology service activities,NaN,NaN,NaN,NaN
3,113 FRIERN ROAD FREEHOLD COMPANY LIMITED,9764999,Active,Private limited company,NaN,NaN,2015-09-07 00:00:00,NaN,NaN,1130,12 Hatherley Road Sidcup DA14 4DT,1130,"Growing of vegetables and melons, roots and tu...",NaN,NaN,NaN,NaN,NaN,NaN
4,1133 LIMITED,14505229,Active,Private limited company,NaN,NaN,2022-11-24 00:00:00,NaN,NaN,01280 47190 85590 96040,40 Morland Estate Richmond Road London E8 3EY,1280,"Growing of spices, aromatic, drug and pharmace...",47190.0,Other retail sale in non-specialised stores,85590.0,Other education n.e.c.,96040.0,Physical well-being activities
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29451,ZULU FARM LIMITED,13954714,Active,Private limited company,NaN,NaN,2022-03-04 00:00:00,NaN,NaN,1500,Carrimers Farm Office Carrimers Farm Aston Tir...,1500,Mixed farming,NaN,NaN,NaN,NaN,NaN,NaN
29452,ZULUBOY LIMITED,14719638,Active,Private limited company,NaN,NaN,2023-03-09 00:00:00,NaN,NaN,01120 47710,6 Crossley Road Manchester M19 1GY,1120,Growing of rice,47710.0,Retail sale of clothing in specialised stores,NaN,NaN,NaN,NaN
29453,ZWETSLOOTS FLORICULTURE LIMITED,12433418,Active,Private limited company,NaN,NaN,2020-01-30 00:00:00,NaN,NaN,1290,"North Lodge Grange Road, Blunham Bedford MK44 3NT",1290,Growing of other perennial crops,NaN,NaN,NaN,NaN,NaN,NaN
29454,ZXLUCKY LIMITED,14724256,Active,Private limited company,NaN,NaN,2023-03-13 00:00:00,NaN,NaN,1460,Ec5m 7hc England 29 Charing Cross Rd London EC...,1460,Raising of swine/pigs,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
sic_code = sic_code.rename(columns={'company_number':"Company Registration No. (1)"})
sic_code

,Owner Name,Company Registration No. (1),company_status,company_type,company_subtype,dissolution_date,incorporation_date,removed_date,registered_date,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
0,@ Cleeve Hill Limited,11723541,Active,Private limited company,NaN,NaN,2018-12-12 00:00:00,NaN,NaN,01500 55900 70229,Upper Lodge Cleeve Hill Kelston Road Bath BA1 9AD,1500,Mixed farming,55900.0,Other accommodation,70229.0,Management consultancy activities other than f...,NaN,NaN
1,1 More Tree Community Interest Company,11038379,Active,Private limited company,Community Interest Company (CIC),NaN,2017-10-30 00:00:00,NaN,NaN,1300,12 Ranelagh Road Wellingborough NN8 1HZ,1300,Plant propagation,NaN,NaN,NaN,NaN,NaN,NaN
2,1 Onion Limited,3354877,Active,Private limited company,NaN,NaN,1997-04-17 00:00:00,NaN,NaN,01250 62090,White Acre Pound Lane Meonstoke SO32 3NP,1250,Growing of other tree and bush fruits and nuts,62090.0,Other information technology service activities,NaN,NaN,NaN,NaN
3,113 Friern Road Freehold Company Limited,9764999,Active,Private limited company,NaN,NaN,2015-09-07 00:00:00,NaN,NaN,1130,12 Hatherley Road Sidcup DA14 4DT,1130,"Growing of vegetables and melons, roots and tu...",NaN,NaN,NaN,NaN,NaN,NaN
4,1133 Limited,14505229,Active,Private limited company,NaN,NaN,2022-11-24 00:00:00,NaN,NaN,01280 47190 85590 96040,40 Morland Estate Richmond Road London E8 3EY,1280,"Growing of spices, aromatic, drug and pharmace...",47190.0,Other retail sale in non-specialised stores,85590.0,Other education n.e.c.,96040.0,Physical well-being activities
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29451,Zulu Farm Limited,13954714,Active,Private limited company,NaN,NaN,2022-03-04 00:00:00,NaN,NaN,1500,Carrimers Farm Office Carrimers Farm Aston Tir...,1500,Mixed farming,NaN,NaN,NaN,NaN,NaN,NaN
29452,Zuluboy Limited,14719638,Active,Private limited company,NaN,NaN,2023-03-09 00:00:00,NaN,NaN,01120 47710,6 Crossley Road Manchester M19 1GY,1120,Growing of rice,47710.0,Retail sale of clothing in specialised stores,NaN,NaN,NaN,NaN
29453,Zwetsloots Floriculture Limited,12433418,Active,Private limited company,NaN,NaN,2020-01-30 00:00:00,NaN,NaN,1290,"North Lodge Grange Road, Blunham Bedford MK44 3NT",1290,Growing of other perennial crops,NaN,NaN,NaN,NaN,NaN,NaN
29454,Zxlucky Limited,14724256,Active,Private limited company,NaN,NaN,2023-03-13 00:00:00,NaN,NaN,1460,Ec5m 7hc England 29 Charing Cross Rd London EC...,1460,Raising of swine/pigs,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
import pandas as pd
import re

def conditional_upper(name):
    if name.isupper():
        return name
    else:
        return name.upper()

# Convert to uppercase


def replace_text(x):
    # Replace brackets with spaces
    pattern = r'[\[\](){}<>]'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_hyph_text(x):
    # Replace hyphens between letters with spaces
    pattern = r'(?<=[a-zA-Z])-(?=[a-zA-Z])'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_and(x):
    # Replace '&' with 'AND'
    return re.sub(r'&', 'AND', x)

def replace_LTD(x):
    # Replace 'LTD' with 'LIMITED'
    return re.sub(r'\bLTD\b', 'LIMITED', x)

def replace_property(x):
    # Replace 'PROPERTY' with 'PROPERTIES'
    return re.sub(r'PROPERTY', 'PROPERTIES', x)  

def remove_space(x):
    # Remove extra spaces
    x = re.sub(r'\s+', ' ', x)
    return x.strip()

def clean_commas(text):
    # Replace commas between words with spaces
    text = re.sub(r'(?<=[a-zA-Z]),(?=[a-zA-Z])', ' ', text)
    # Remove any trailing commas or trailing spaces
    text = re.sub(r',\s*$', '', text)
    # Remove leading or consecutive commas
    text = re.sub(r'(^,|,{2,})', '', text)
    # Remove extra spaces around commas
    text = re.sub(r'\s*,\s*', ' ', text)
    return text.strip()


def replace_special_characters_and_normalize(name):
    # Replace periods and hyphens with spaces
    replaced_chars = re.sub(r'[.-]', ' ', name)
    # Remove leading, trailing, and multiple spaces
    cleaned_name = re.sub(r'\s+', ' ', replaced_chars).strip()
    # Capitalize each word properly
    normalized_name = ' '.join(word.capitalize() for word in cleaned_name.split())
    return normalized_name

def remove_after_limited(x):
    pattern = r'(Limited).*'  # Match "Limited" and everything after
    return re.sub(pattern, r'\1', x, flags=re.IGNORECASE)


# Apply the functions to the 'company_na' column



sic_code['Owner Name'] = sic_code['Owner Name'].apply(conditional_upper) 
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_text)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_hyph_text)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_LTD)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_and)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_property)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(replace_special_characters_and_normalize)
sic_code["Owner Name"] = sic_code["Owner Name"].apply(remove_space)
sic_code['Owner Name'] = sic_code['Owner Name'].apply(clean_commas)
sic_code['Owner Name'] = sic_code['Owner Name'].apply(remove_after_limited)
sic_code['Owner Name'] = sic_code['Owner Name'].str.replace(':', '').str.strip()

sic_code['Owner Name'] = sic_code['Owner Name'].str.upper()
sic_code['Owner Name'] = sic_code['Owner Name'].str.title()



In [24]:
sic_code

,farm_name,company_number,company_status,company_type,company_subtype,dissolution_date,incorporation_date,removed_date,registered_date,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
0,@ Cleeve Hill Limited,11723541,Active,Private limited company,NaN,NaN,2018-12-12 00:00:00,NaN,NaN,01500 55900 70229,Upper Lodge Cleeve Hill Kelston Road Bath BA1 9AD,1500,Mixed farming,55900.0,Other accommodation,70229.0,Management consultancy activities other than f...,NaN,NaN
1,1 More Tree Community Interest Company,11038379,Active,Private limited company,Community Interest Company (CIC),NaN,2017-10-30 00:00:00,NaN,NaN,1300,12 Ranelagh Road Wellingborough NN8 1HZ,1300,Plant propagation,NaN,NaN,NaN,NaN,NaN,NaN
2,1 Onion Limited,3354877,Active,Private limited company,NaN,NaN,1997-04-17 00:00:00,NaN,NaN,01250 62090,White Acre Pound Lane Meonstoke SO32 3NP,1250,Growing of other tree and bush fruits and nuts,62090.0,Other information technology service activities,NaN,NaN,NaN,NaN
3,113 Friern Road Freehold Company Limited,9764999,Active,Private limited company,NaN,NaN,2015-09-07 00:00:00,NaN,NaN,1130,12 Hatherley Road Sidcup DA14 4DT,1130,"Growing of vegetables and melons, roots and tu...",NaN,NaN,NaN,NaN,NaN,NaN
4,1133 Limited,14505229,Active,Private limited company,NaN,NaN,2022-11-24 00:00:00,NaN,NaN,01280 47190 85590 96040,40 Morland Estate Richmond Road London E8 3EY,1280,"Growing of spices, aromatic, drug and pharmace...",47190.0,Other retail sale in non-specialised stores,85590.0,Other education n.e.c.,96040.0,Physical well-being activities
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29451,Zulu Farm Limited,13954714,Active,Private limited company,NaN,NaN,2022-03-04 00:00:00,NaN,NaN,1500,Carrimers Farm Office Carrimers Farm Aston Tir...,1500,Mixed farming,NaN,NaN,NaN,NaN,NaN,NaN
29452,Zuluboy Limited,14719638,Active,Private limited company,NaN,NaN,2023-03-09 00:00:00,NaN,NaN,01120 47710,6 Crossley Road Manchester M19 1GY,1120,Growing of rice,47710.0,Retail sale of clothing in specialised stores,NaN,NaN,NaN,NaN
29453,Zwetsloots Floriculture Limited,12433418,Active,Private limited company,NaN,NaN,2020-01-30 00:00:00,NaN,NaN,1290,"North Lodge Grange Road, Blunham Bedford MK44 3NT",1290,Growing of other perennial crops,NaN,NaN,NaN,NaN,NaN,NaN
29454,Zxlucky Limited,14724256,Active,Private limited company,NaN,NaN,2023-03-13 00:00:00,NaN,NaN,1460,Ec5m 7hc England 29 Charing Cross Rd London EC...,1460,Raising of swine/pigs,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
merged_df = pd.merge(data, sic_code, on='Company Registration No. (1)', how='left')
merged_df

,Unnamed: 0,OWNER_ID,Owner Name_x,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,...,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
0,0,OWNER00001,001Hardy Limited,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,OWNER00002,07737281 Limited,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,OWNER00003,1 Grosvenor Holdings Limited,10332472,"263 Dunstable Road, Luton LU4 8BP",BEDFORD,BEDFORD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,OWNER00004,106 Properties Group Limited,9789625,"1-2 The Barn, West Stoke Road, West Lavant, Ch...",CHICHESTER,WEST SUSSEX,PO20 1LJ,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,OWNER00005,112680647,NaN,NaN,NaN,NaN,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58000,58000,OWNER58001,Zurich Developments Limited,9133879,"246 Hawkes Mill Lane, Allesley, Coventry CV5 9FJ",COVENTRY,WEST MIDLANDS,CV5 9FJ,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58001,58001,OWNER58002,Zym Capital Limited,14747989,"Aquinas House, 63 Warstone Lane, Birmingham B1...",STRATFORD-ON-AVON,WARWICKSHIRE,B94 5DT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58002,58002,OWNER58003,Zymshire Limited,NaN,NaN,NaN,NaN,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58003,58003,OWNER58004,Zyox Limited,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
merged_df  = merged_df[['ID', 'farm_name', 'new_lat', 'new_lon', 'New_full_a', 'Full Post',
       'Place type', 'Herbs & Or', 'Other crop', 'Permanent', 'Temporary',
       'Vegetables', 
       'nature_of_business', 'registered_office_address',
       'nature_of_business1', 'nature_of_business1_name',
       'nature_of_business2', 'nature_of_business2_name',
       'nature_of_business3', 'nature_of_business3_name',
       'nature_of_business4', 'nature_of_business4_name', 'Area(hec)', 'crop_maxA',]]

merged_df

,ID,farm_name,new_lat,new_lon,New_full_a,Full Post,Place type,Herbs & Or,Other crop,Permanent,...,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name,Area(hec),crop_maxA
0,FARM00008,"""WINDWARD"" REYNOLDS LANE",52.3091,-2.3517,"ABBERLEY, WORCESTER, WR6 6BT, UNITED KINGDOM",WR6 6BT,Farm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.469178,"['Spring Wheat', 'Spring Barley', 'Grass', 'Sp..."
1,FARM00009,(EGG LADY) DOUBLE YOLKERS EGGS,53.4003,-0.5729,"WATER TOWER COMPLEX, BARTON RD, CAENBY CORNER ...",DN21 5TN,Egg supplier,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.621334,"['Spring Field beans', 'Winter Oilseed', 'Fall..."
2,FARM00004,. RAW MILK DUMELOWSDAIRY,52.7945,-1.4487,"THE GABLES FARM MAIN STREET, CALKE, LEICESTER,...",LE65 1RR,Dairy farm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.810473,['Fallow Land']
3,FARM00013,1 1 ROOKERY HOUSE GROVE FARM,51.2674,-0.8564,"1 1 ROOKERY HOUSE GROVE FARM, 1 1 ROOKERY HOUS...",GU51 5RX,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.748515,"['Potato', 'Maize']"
4,FARM00014,1 ABBEY FARM,54.6854,-2.6124,"1 ABBEY FARM, 1 ABBEY FARM HOUSE SKIRWITH PENR...",CA10 1RB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.714574,"['Spring Wheat', 'Spring Barley', 'Beet', 'Gra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59166,FARM57812,WHATTON APIARIES,52.8139,-1.2771,NaN,NaN,Farm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166148,['Winter Field beans']
59167,FARM58890,WILLOW WOOD FARM,50.7486,-1.5503,NaN,NaN,Farm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.907263,"['Spring Barley', 'Spring Field beans', 'Grass..."
59168,FARM60530,WOOLACOMBE FARM,51.1726,-4.2066,NaN,NaN,Mixed Farming,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.557985,['Grass']
59169,FARM60134,WRAGBY FARM,53.2863,-0.3002,NaN,NaN,Livestock Breeders,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.137807,"['Trees and Scrubs, short Woody plants, hedger..."


In [30]:
merged_df.to_excel("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/farm_voronoi/farm_wise/Farm_base_wise_with_nature_of_business.xlsx")

In [16]:
merged_df.loc[21]

Unnamed: 0                                                                     21
OWNER_ID                                                               OWNER00022
Owner Name                                                  2 Agriculture Limited
Company Registration No. (1)                                             SC156515
Proprietor (1) Address (1)      Fairview Mill, Ingleston, Newbridge, Midlothia...
District                                             KING'S LYNN AND WEST NORFOLK
County                                                                    NORFOLK
Postcode                                                                      NaN
Proprietor Name (2)                                                           NaN
Country_side                                                                  NaN
name_from_LR                                                                  NaN
Area(hec)                                                                5.591334
crop_maxA       

In [42]:
merged_df[~merged_df['Owner Name_y'].isna()]

,OWNER_ID,Owner Name_x,Owner Name_y,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,...,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
21,OWNER00022,2 Agriculture Limited,2 Agriculture Limited,SC156515,"Fairview Mill, Ingleston, Newbridge, Midlothia...",KING'S LYNN AND WEST NORFOLK,NORFOLK,NaN,NaN,NaN,...,01470 10910,Fairview Mill Ingliston Newbridge EH28 8NB,1470.0,Raising of poultry,10910.0,Manufacture of prepared feeds for farm animals,NaN,NaN,NaN,NaN
1829,OWNER01830,Albert Bartlett And Sons Airdrie Limited,Albert Bartlett And Sons Airdrie Limited,SC037896,"New Monkland, 251 Stirling Road, Airdrie ML6 7SP",BOSTON,LINCOLNSHIRE,PE20 1TT,NaN,NaN,...,01130 10310 10850 46310,New Monkland 251 Stirling Road Airdrie ML6 7SP,1130.0,"Growing of vegetables and melons, roots and tu...",10310.0,Processing and preserving of potatoes,10850.0,Manufacture of prepared meals and dishes,46310.0,Wholesale of fruit and vegetables
4538,OWNER04539,Belmont Farms Limited,Belmont Farms Limited,NI025971,"Wellington House, 30 Darling Street, Ennisille...",EAST LINDSEY,LINCOLNSHIRE,NaN,NaN,NaN,...,1460,Wellington House 30 Darling Street Enniskillen...,1460.0,Raising of swine/pigs,NaN,NaN,NaN,NaN,NaN,NaN
5805,OWNER05806,Braishfield Manor Estate Limited,Braishfield Manor Estate Limited,SC059221,"39 George Street, Edinburgh EH2 2HZ",TEST VALLEY,HAMPSHIRE,SO51 0PS,NaN,Y,...,01110 68201,Innerwick Ferntower Road Crieff Perth & Kinros...,1110.0,"Growing of cereals (except rice), leguminous c...",68201.0,Renting and operating of Housing Association r...,NaN,NaN,NaN,NaN
7783,OWNER07784,Cambusmore Estates Limited,Cambusmore Estates Limited,SC227042,"Cambusmore Lodge, Dornoch IV25 3JD",TORRIDGE,DEVON,NaN,NaN,Y,...,01450 01700 02100 03120,Cambusmore Lodge Dornoch Sutherland IV25 3JD,1450.0,Raising of sheep and goats,1700.0,"Hunting, trapping and related service activities",2100.0,Silviculture and other forestry activities,3120.0,Freshwater fishing
20875,OWNER20876,Hayes Farms,Hayes Farms,NI068059,"110 Moy Road, Dungannon BT71 7DX",WESTMORLAND AND FURNESS,WESTMORLAND AND FURNESS,CA11 9NW,NaN,NaN,...,1500,110 Moy Road Dungannon BT71 7DX,1500.0,Mixed farming,NaN,NaN,NaN,NaN,NaN,NaN
22456,OWNER22457,Howden Enterprises Limited,Howden Enterprises Limited,NI635402,"118 Trew Mount Road, Dungannon, County Tyrone ...",EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,NaN,NaN,NaN,...,1130,118 Trew Mount Road Dungannon BT71 7EF,1130.0,"Growing of vegetables and melons, roots and tu...",NaN,NaN,NaN,NaN,NaN,NaN
41976,OWNER41977,R C Reed Limited,R C Reed Limited,SC100435,"Norham East Mains, Norham, Berwick-Upon-Tweed,...",NORTHUMBERLAND,NORTHUMBERLAND,NaN,NaN,NaN,...,1110,Exchange Place 3 Semple Street Edinburgh EH3 8BL,1110.0,"Growing of cereals (except rice), leguminous c...",NaN,NaN,NaN,NaN,NaN,NaN
48852,OWNER48853,Sunny Side Eggs Limited,Sunny Side Eggs Limited,SC769310,"St. Johns Kirk, Symington, Biggar ML12 6JU",LANCASTER,LANCASHIRE,LA6 1BQ,NaN,Y,...,1470,St. Johns Kirk Symington Biggar ML12 6JU,1470.0,Raising of poultry,NaN,NaN,NaN,NaN,NaN,NaN
49310,OWNER49311,T C Beattie And Co Limited,T C Beattie And Co Limited,SC296896,"Wyseby Mains, Kirtlebridge, Lockerbie, Dumfrie...",CUMBERLAND,CUMBERLAND,CA6 4HG,NaN,NaN,...,1410,Wyseby Mains Kirtlebridge Lockerbie DG11 3AW,1410.0,Raising of dairy cattle,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
merged_df = merged_df[['OWNER_ID', 'Owner Name_x','Owner Name_y',
       'Company Registration No. (1)', 'Proprietor (1) Address (1)',
       'District', 'County', 'Postcode', 'Proprietor Name (2)', 'Country_side',
       'name_from_LR', 'Area(hec)', 'crop_maxA', 
       'company_status', 'company_type', 'company_subtype', 'dissolution_date',
       'incorporation_date', 'removed_date', 'registered_date',
       'nature_of_business', 'registered_office_address',
       'nature_of_business1', 'nature_of_business1_name',
       'nature_of_business2', 'nature_of_business2_name',
       'nature_of_business3', 'nature_of_business3_name',
       'nature_of_business4', 'nature_of_business4_name']]
merged_df

,OWNER_ID,Owner Name_x,Owner Name_y,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,...,nature_of_business,registered_office_address,nature_of_business1,nature_of_business1_name,nature_of_business2,nature_of_business2_name,nature_of_business3,nature_of_business3_name,nature_of_business4,nature_of_business4_name
0,OWNER00001,001Hardy Limited,NaN,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OWNER00002,07737281 Limited,NaN,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OWNER00003,1 Grosvenor Holdings Limited,NaN,10332472,"263 Dunstable Road, Luton LU4 8BP",BEDFORD,BEDFORD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OWNER00004,106 Properties Group Limited,NaN,9789625,"1-2 The Barn, West Stoke Road, West Lavant, Ch...",CHICHESTER,WEST SUSSEX,PO20 1LJ,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OWNER00005,112680647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58000,OWNER58001,Zurich Developments Limited,NaN,9133879,"246 Hawkes Mill Lane, Allesley, Coventry CV5 9FJ",COVENTRY,WEST MIDLANDS,CV5 9FJ,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58001,OWNER58002,Zym Capital Limited,NaN,14747989,"Aquinas House, 63 Warstone Lane, Birmingham B1...",STRATFORD-ON-AVON,WARWICKSHIRE,B94 5DT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58002,OWNER58003,Zymshire Limited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58003,OWNER58004,Zyox Limited,NaN,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
